In [ ]:
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes

!pip install pytesseract evaluate tqdm rouge-score accelerate nltk tensorboard jupyter-black py7zr --upgrade
!apt-get install git --yes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-da

In [ ]:
import torch
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    RobertaModel,
)
import json
import pandas
import jupyter_black
from datetime import timedelta
from datasets import Dataset
model_id = "roberta-base"

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

In [ ]:
with open("data.json", "r") as f:
    data = json.load(f)

# create a dataframe
df = pandas.DataFrame(data)

dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1)

train_dataset = dataset['train']
test_dataset = dataset["test"]
val_dataset = dataset['test']

print("train_dataset")
print(train_dataset)
print("test_dataset")
print(test_dataset)
print("val_dataset")
print(val_dataset)

train_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score'],
    num_rows: 5
})
test_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score'],
    num_rows: 1
})
val_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score'],
    num_rows: 1
})


In [ ]:
def adjust_dataset(example):
  line_break = '\n'
  size = len(example['id'])

  prompt = [
      f"""
        {example['task'][i]}

        Question:
        {example['question'][i]}

        Maximum score:
        {example['maximum_score'][i]}

        Scoring guide:
        {line_break.join(str(str(z['point']) + ' points - ' + z['criteria']) for z in example['scoring_guide'][i])}

        Answer:
        {example['answer'][i]}

        Score:
        <mask>
      """
      for i in range(size)]

  example["text"] = prompt
  example["label"] = example["score"]

  #for i in range(len(example["score"])):
  #  example["label"][i] = str(example["score"][i])

  return example

train_dataset = train_dataset.map(adjust_dataset, batched=True)
test_dataset = test_dataset.map(adjust_dataset, batched=True)
val_dataset = val_dataset.map(adjust_dataset, batched=True)

print("train_dataset")
print(train_dataset)
print("test_dataset")
print(test_dataset)
print("val_dataset")
print(val_dataset)



Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

train_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label'],
    num_rows: 5
})
test_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label'],
    num_rows: 1
})
val_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label'],
    num_rows: 1
})


In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

print("train_dataset")
print(train_dataset)
print("test_dataset")
print(test_dataset)
print("val_dataset")
print(val_dataset)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

train_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5
})
test_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1
})
val_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1
})


In [ ]:
# Set dataset format
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

print("train_dataset")
print(train_dataset)
print("test_dataset")
print(test_dataset)
print("val_dataset")
print(val_dataset)

train_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5
})
test_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1
})
val_dataset
Dataset({
    features: ['id', 'task', 'question', 'maximum_score', 'scoring_guide', 'answer', 'score', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1
})


In [ ]:
# We will need this to directly output the class names when using the pipeline without mapping the labels later.
# Extract the number of classes and their names
num_labels = 7
class_names = [0, 1, 2, 3, 4, 5, 6]
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

# Update the model's configuration with the id2label mapping
my_config = AutoConfig.from_pretrained(model_id)
my_config.update({"id2label": id2label})

number of labels: 7
the labels: ['0', '1', '2', '3', '4', '5', '6']


In [ ]:
original_model = RobertaModel.from_pretrained(model_id, config=my_config)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
output_dir = f'./first-ever-training'

# TrainingArguments
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1,

)

# Trainer
trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

ValueError: ignored